In [341]:
from IPython.display import display
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [342]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import PY
from cafle.genfunc import EmptyClass

In [343]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction

* Logistic center development

# Initial Setting

In [344]:
astn = EmptyClass()

# Input Index Data

In [345]:
tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-12', 30],
           ['cstrn', '2022-01', 22],
           ['loan',  '2022-01', 26],
           ['sales', '2022-01', 26]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start   = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq    = 'M')

# Input Financing Condition Data

In [346]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl']
tmp_ary = [['equity',  1_300,     1_300]]
astn.equity = DataFrame(tmp_ary, columns=tmp_col)

equity = cf.Loan(idx,
                 amt_ntnl = astn.equity.amt_ntnl[0],
                 amt_intl = astn.equity.amt_intl[0])

In [347]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl', 'rate_fee', 'rate_IR']
tmp_ary = [['tra',    65_000,     15_000,      0.015,     0.040],
           ['trb',    20_000,     20_000,      0.035,     0.060],
           ['trc',    10_000,     10_000,      0.070,     0.090]]
astn.loan = DataFrame(tmp_ary, columns=tmp_col)

loan = cf.Intlz_loan(idx, idx.loan,
                     title = astn.loan.title,
                     amt_ntnl = astn.loan.amt_ntnl,
                     amt_intl = astn.loan.amt_intl,
                     rate_fee = astn.loan.rate_fee,
                     rate_IR = astn.loan.rate_IR)

print("## Financing Condition ##\n")
print(f"Total Equity amount:{sum(astn.equity.amt_ntnl):>7,.0f}")
print(f"Total loan amount:  {sum(astn.loan.amt_ntnl):>7,.0f}")
print(R(astn.loan, rate=['rate_fee', 'rate_IR']))

## Financing Condition ##

Total Equity amount:  1,300
Total loan amount:   95,000
  title amt_ntnl amt_intl rate_fee rate_IR
0   tra   65,000   15,000      1.5     4.0
1   trb   20,000   20,000      3.5     6.0
2   trc   10,000    5,000      7.0     9.0


# Input Sales Data

In [348]:
# Pickling Data
with open("data/valuation.pickle", "rb") as fr:
    data_valuation = pickle.load(fr)
valuation = data_valuation['valuation_pivot']
value = valuation.loc[[('23,000', '50,000')]].values[0]
print("## Valuation Matrix ##")
display(R(valuation))

## Valuation Matrix ##


cap_rate,"(4.0, 4.6)","(4.1, 4.8)","(4.2, 5.0)","(4.3, 5.2)","(4.4, 5.4)"
rent,,,,,
"(20,000, 44,000)","118,480","114,366","110,543","106,980","103,651"
"(21,000, 46,000)","123,728","119,432","115,441","111,721","108,246"
"(22,000, 48,000)","128,976","124,499","120,339","116,463","112,840"
"(23,000, 50,000)","134,224","129,566","125,238","121,204","117,435"
"(24,000, 52,000)","139,472","134,633","130,136","125,945","122,029"
"(25,000, 54,000)","144,720","139,700","135,034","130,687","126,624"


In [349]:
sales_value = 125_000
sales = cf.Account(idx, 'Sales')
sales.addamt(idx.sales[0], sales_value)
sales.subscdd(idx.sales[-1], sales_value)

# Input Cost Data

### 1) Set Cost Class

In [350]:
cost = cf.Cost(idx)

### 2) Land Purchase Costs

In [351]:
title = ('lnd', 'prchs') # land purchase cost
byname = '용지매입비'
tmp_ary = [[idx.locval(2021, 12),  1_000],
           [idx.locval(2022,  1), 13_500]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [352]:
title = ('lnd', 'brkrg') # land brokerage fee
byname = '부동산중개비용'
tmp_ary = [[idx.locval(2021, 12),  39],
           [idx.locval(2022,  1),  91]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [353]:
title = ('lnd', 'tax_aqstn') # land acquisition and registration tax
byname = '취등록세'
tmp_ary = [[idx.locval(2022, 1), 675]]
# 토지매매대금 x 취득세 4.0%, 농특세 0.2%, 교육세 0.4%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [354]:
title = ('lnd', 'nhbond') # national housing bond
byname = '국민주택채권'
tmp_ary = [[idx.locval(2022, 1), 36]]
# 토지매매대금 x 매입률 4.5% x 본인부담율 5.5%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [355]:
title = ('lnd', 'jdclcst') # judicial scrivener cost
byname = '법무사비용'
tmp_ary = [[idx.locval(2022, 1), 15]]
# 토지매매대금 x 0.1%
cost.inptcst(title, byname, scdddf=tmp_ary)

### 3) Construction Costs

In [356]:
cstrn = {}
cstrn['dmltn'] = 35 + 1_180 # 철거비: 3,550평 x 342천원, 온천공 폐공공사 포함
cstrn['civil'] = 8_490 # 토목공사: 16,327평 x 520천원
cstrn['nml_fclt'] = 9_311 * 2.66 # 상온시설: 9,311평 x 2,660천원
cstrn['cld_fclt'] = 8_365 * 3.66 # 저온시설: 8,365평 x 3,660천원
cstrn_sum = round(sum([cstrn[key] for key in cstrn]))

retention_rate = 0.1
cstrn_intl = 35
cstrn_prd = cstrn_sum * (1-retention_rate) - cstrn_intl
cstrn_rttn = cstrn_sum * retention_rate

print(f"Total construction cost:   {cstrn_sum:>7,.0f}")
print(f"Initial construction cost: {cstrn_intl:>7,.0f}")
print(f"Period construction cost:  {cstrn_prd:>7,.0f}")
print(f"Retained construction cost:{cstrn_rttn:>7,.0f}")

Total construction cost:    65,088
Initial construction cost:      35
Period construction cost:   58,544
Retained construction cost:  6,509


In [357]:
title = ('cstrn', 'cstrn') # construction cost
byname = '공사비'
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[idx.prjt[0], cstrn_intl]]
tmp_ary.extend([[x, cstrn_prd / tmp_len] for x in tmp_idx])
tmp_ary.append([idx.prjt[-1], cstrn_rttn])

cost.inptcst(title, byname, scdddf=tmp_ary)

### 4) Indirect Construction Costs

In [358]:
title = ('cstrnidrt', 'wtrelec') # water supply, electricity etc. 
byname = '각종인입비'
tmp_ary = [[idx.locval(2023, 10), 980]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [359]:
title = ('cstrnidrt', 'wsctbn') # water and sewage contribution
byname = '상수도분담금'
tmp_ary = [[idx.locval(2023, 10), 163]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [360]:
title = ('cstrnidrt', 'dsgncst') # design cost
byname = '설계비'
tmp_ary = [[idx.locval(2022, 1), 624]]
# 16,327평 x 38천원
cost.inptcst(title, byname, scdddf=tmp_ary)

In [361]:
title = ('cstrnidrt', 'lcnscst') # license cost
byname = '인허가비용'
tmp_ary = [[idx.locval(2022, 1), 528]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [362]:
title = ('cstrnidrt', 'spvsn') # supervision cost
byname = '감리비'
ttl_amt = 490 # 16,327평 x 30천원, 월별 분할 부과
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 5) Marketing Costs

In [363]:
title = ('mrktg', 'rentagncy') # rent agency cost
byname = '임대대행수수료'
tmp_ary = [[idx.locval(2023, 3), 517],
           [idx.locval(2023, 4), 517],
           [idx.locval(2023, 5), 517]]
# 임대계약 실행시 월 임대료의 3개월치 지급
# 1개월치 임대료 517백만원 가정
cost.inptcst(title, byname, scdddf=tmp_ary)

In [364]:
title = ('mrktg', 'salescsltg') # sales consulting fee
byname = '매각컨설팅수수료'
tmp_ary = [[idx.locval(2023, 12), 967]]
# 추정매각가 x 0.8%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [365]:
title = ('mrktg', 'advtsmnt') # advertisement and promotion cost
byname = '광고홍보비'
tmp_ary = [[idx.locval(2022, 6), 250],
           [idx.locval(2023, 6), 250]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 6) Tax and Utility bills(제세공과금)

In [366]:
title = ('taxutlt', 'prpttax') # property tax
byname = '재산세종부세'
tmp_ary = [[idx.locval(2022, 6), 37],
           [idx.locval(2023, 6), 37]]
# 5,590 x 0.62% 2년
cost.inptcst(title, byname, scdddf=tmp_ary)

In [367]:
title = ('taxutlt', 'prsvtntax') # preservation registration fee
byname = '보존등기비'
tmp_ary = [[idx.cstrn.index[-1], 3_071]]
# 건물 취득비용 x 3.40%(취득세 2.8%, 농특세 0.2%, 교육세 0.16%, 법무사 0.24%)
cost.inptcst(title, byname, scdddf=tmp_ary)

### 7) Additional Costs

In [368]:
title = ('addtnl', 'pmfee') # Additional expense, PM fee
byname = 'PM수수료'
tmp_ary = [[idx.locval(2021, 12), 200]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [369]:
title = ('addtnl', 'oprtgcst') # company operating cost
unt_amt = 20 # 2,000만원/월
tmp_idx = idx.loan.index
tmp_ary = [[x, unt_amt] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [370]:
title = ('addtnl', 'rsrvfund') # reserve fund
byname = '예비비'
tmp_ary = [[idx.locval(2022, 6), 907],
           [idx.locval(2023, 6), 907]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [371]:
title = ('addtnl', 'trustcst') # trust fee
byname = '신탁수수료'
tmp_ary = [[idx.locval(2022, 1), 967]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [372]:
title = ('addtnl', 'csltcst') # consulting fee
byname = '사평감평용역비'
tmp_ary = [[idx.locval(2022, 1), 10],
           [idx.locval(2022, 1), 140]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [373]:
title = ('addtnl', 'legalcst') # legal advice fee
byname = '법률자문'
tmp_ary = [[idx.locval(2022, 1), 40]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 8) Financing Costs

In [374]:
title = ('fncg', 'arngmnt') # financing arrangement fee
byname = '금융주관수수료'
tmp_rate = 0.02 #2.0%
tmp_amt = sum(loan.amt_ntnl)
tmp_ary = [[idx.locval(2022, 1), tmp_amt * tmp_rate]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [375]:
title = ('fncg', 'spcoprtg') # spc operating cost
byname = 'SPC유동화비용'
tmp_ary = [[idx.locval(2022, 1), 66]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [376]:
title = ('fncg', 'agntbank') # agent banking fee
byname = '대리금융기관'
tmp_ary = [[idx.locval(2022, 1), 30],
           [idx.locval(2023, 1), 30]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 9) Cost Matrix

In [410]:
cost_dct = cost["all"].dct.items()
cost_df = DataFrame({key:item.add_scdd[:] for key, item in cost_dct})
cost_df[("sum", "sum")] = cost["all"].add_scdd[:]
display(R(cost_df))

lnd                                 cstrn cstrnidrt         \
             prchs brkrg tax_aqstn nhbond jdclcst  cstrn   wtrelec wsctbn   
2021-12-31   1,000  39.0         0    0.0     0.0     35         0      0   
2022-01-31  13,500  91.0       675   36.0    15.0  2,545         0      0   
2022-02-28       0   0.0         0    0.0     0.0  2,545         0      0   
2022-03-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-04-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-05-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-06-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-07-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-08-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-09-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-10-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-11-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-12-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-01-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-02-28       0   0.0         0    0.0     0.0  2,545         0      0   
2023-03-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-04-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-05-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-06-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-07-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-08-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-09-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-10-31       0   0.0         0    0.0     0.0  2,545       980    163   
2023-11-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-12-31       0   0.0         0    0.0     0.0      0         0      0   
2024-01-31       0   0.0         0    0.0     0.0      0         0      0   
2024-02-29       0   0.0         0    0.0     0.0      0         0      0   
2024-03-31       0   0.0         0    0.0     0.0      0         0      0   
2024-04-30       0   0.0         0    0.0     0.0      0         0      0   
2024-05-31       0   0.0         0    0.0     0.0      0         0      0   
2024-06-30       0   0.0         0    0.0     0.0  6,509         0      0   

                                     mrktg                     taxutlt  \
           dsgncst lcnscst spvsn rentagncy salescsltg advtsmnt prpttax   
2021-12-31       0       0   0.0         0          0        0     0.0   
2022-01-31     624     528  21.3         0          0        0     0.0   
2022-02-28       0       0  21.3         0          0        0     0.0   
2022-03-31       0       0  21.3         0          0        0     0.0   
2022-04-30       0       0  21.3         0          0        0     0.0   
2022-05-31       0       0  21.3         0          0        0     0.0   
2022-06-30       0       0  21.3         0          0      250    37.0   
2022-07-31       0       0  21.3         0          0        0     0.0   
2022-08-31       0       0  21.3         0          0        0     0.0   
2022-09-30       0       0  21.3         0          0        0     0.0   
2022-10-31       0       0  21.3         0          0        0     0.0   
2022-11-30       0       0  21.3         0          0        0     0.0   
2022-12-31       0       0  21.3         0          0        0     0.0   
2023-01-31       0       0  21.3         0          0        0     0.0   
2023-02-28       0       0  21.3         0          0        0     0.0   
2023-03-31       0       0  21.3       517          0        0     0.0   
2023-04-30       0       0  21.3       517          0        0     0.0   
2023-05-31       0       0  21.3       517          0        0     

In [411]:
cost_df.sum()

lnd        prchs         14500.0
           brkrg           130.0
           tax_aqstn       675.0
           nhbond           36.0
           jdclcst          15.0
cstrn      cstrn         65088.0
cstrnidrt  wtrelec         980.0
           wsctbn          163.0
           dsgncst         624.0
           lcnscst         528.0
           spvsn           490.0
mrktg      rentagncy      1551.0
           salescsltg      967.0
           advtsmnt        500.0
taxutlt    prpttax          74.0
           prsvtntax      3071.0
addtnl     pmfee           200.0
           oprtgcst        540.0
           rsrvfund       1814.0
           trustcst        967.0
           csltcst         150.0
           legalcst         40.0
fncg       arngmnt        1900.0
           spcoprtg         66.0
           agntbank         60.0
sum        sum           95129.0
dtype: float64

# Execution Cash Flow

In [379]:
acc = cf.Intlz_accounts(idx, ['oprtg', 'sales', 'repay'])

In [380]:
# Execute cash flow
for idxno in idx.index:
    #### Loans: set loan withdrawble ####
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno, idx[0])
    loan.tra.set_wtdrbl_intldate(idxno)
    loan.trb.set_wtdrbl_intldate(idxno)
    loan.trc.set_wtdrbl_intldate(idxno)
    
    #### Cash Inflow: cash inflow from sales or rent etc. ####
    salesamt = sales.sub_scdd[idxno]
    if salesamt > 0:
        sales.send(idxno, salesamt, acc.repay)
    
    #### Expected Costs: calculate expected costs ####
    # calculate operating costs
    oprtg_cost = cost['all'].add_scdd[idxno]

    # calculate financial costs
    for trch, loaneach in loan.dct.items():
        if idxno == loaneach.idxfn[0]:
            loaneach.fee.addscdd(idxno, loaneach.fee.amt)
        if all([loaneach.is_wtdrbl, not loaneach.is_repaid]):
            #tmp_ntnl = -loaneach.ntnl.bal_strt[idxno]
            #tmp_IRamt = tmp_ntnl * loaneach.IR.rate_cycle
            loaneach.IR.addscdd(idxno, loaneach.IRamt_topay(idxno))
    fncl_fee = loan.ttl.fee.add_scdd[idxno]
    fncl_IR = loan.ttl.IR.add_scdd[idxno]

    cost_ttl = oprtg_cost + fncl_fee + fncl_IR
    
    
    #### Loans: withdraw loan ####
    # calculate the amount to withdraw
    amt_rqrd = acc.oprtg.amt_rqrd_excess(idxno, cost_ttl)
    
    # withdraw loan amount
    amt_wtdrw = 0
    amt_wtdrw += equity.wtdrw(idxno, equity.amt_intl, acc.oprtg)
    if idxno == idx.loan[0]:
        amt_wtdrw += loan.trc.wtdrw(idxno, loan.trc.amt_intl, acc.oprtg)
        amt_wtdrw += loan.trb.wtdrw(idxno, loan.trb.amt_intl, acc.oprtg)
        amt_wtdrw += loan.tra.wtdrw(idxno, loan.tra.amt_intl, acc.oprtg)
    
    amt_rqrd = max(amt_rqrd - amt_wtdrw, 0)
    amt_rqrd = max(amt_rqrd - loan.trc.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)
    amt_rqrd = max(amt_rqrd - loan.trb.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)
    amt_rqrd = max(amt_rqrd - loan.tra.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)

    
    #### Costs: 토지비, 공사비 등 각종 비용 지출 ####
    for cst_name, cst_acc in cost['all'].dct.items():
        amt_scdd = cst_acc.add_scdd[idxno]
        acc.oprtg.send(idxno, amt_scdd, cst_acc)
        
    
    #### Loans: pay financial cost ####
    acc.oprtg.send(idxno, loan.tra.fee.add_scdd[idxno], loan.tra.fee)
    acc.oprtg.send(idxno, loan.trb.fee.add_scdd[idxno], loan.trb.fee)
    acc.oprtg.send(idxno, loan.trc.fee.add_scdd[idxno], loan.trc.fee)
    
    acc.oprtg.send(idxno, loan.tra.IR.add_scdd[idxno], loan.tra.IR)
    acc.oprtg.send(idxno, loan.trb.IR.add_scdd[idxno], loan.trb.IR)
    acc.oprtg.send(idxno, loan.trc.IR.add_scdd[idxno], loan.trc.IR)
    
    
    #### Loans: repay loan amount ####
    if idxno >= loan.idxfn[-1]: # 만기 도래 여부 확인
        amtrpy = loan.tra.repay_amt(idxno, acc.repay.bal_end[idxno])
        acc.repay.send(idxno, amtrpy, loan.tra.ntnl)
        loan.tra.set_repaid(idxno)
                
        if loan.tra.is_repaid:
            amtrpy = loan.trb.repay_amt(idxno, acc.repay.bal_end[idxno])
            acc.repay.send(idxno, amtrpy, loan.trb.ntnl)
            loan.trb.set_repaid(idxno)
            
        if loan.trb.is_repaid:
            amtrpy = loan.trc.repay_amt(idxno, acc.repay.bal_end[idxno])
            acc.repay.send(idxno, amtrpy, loan.trc.ntnl)
            loan.trc.set_repaid(idxno)
            
        if loan.trc.is_repaid:
            acc.repay.send(idxno, acc.repay.bal_end[idxno], acc.oprtg)
    
    #### Loans: Set back loan unwithdrawble at maturity ####
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    loan.tra.setback_wtdrbl_mtrt(idxno)
    loan.trb.setback_wtdrbl_mtrt(idxno)
    loan.trc.setback_wtdrbl_mtrt(idxno)

# Analysis of Results

### 1) Accounts

In [412]:
print("## Operating Account ##")
display(R(acc.oprtg.df))

## Operating Account ##


,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.0,0.0,0,"1,300","1,274",26
2022-01-31,0.0,0.0,26,"40,000","23,584","16,442"
2022-02-28,0.0,0.0,"16,442",0,"2,774","13,668"
2022-03-31,0.0,0.0,"13,668",0,"2,774","10,894"
2022-04-30,0.0,0.0,"10,894",0,"2,774","8,120"
2022-05-31,0.0,0.0,"8,120",0,"2,774","5,345"
2022-06-30,0.0,0.0,"5,345",0,"3,968","1,377"
2022-07-31,0.0,0.0,"1,377","1,400","2,774",3
2022-08-31,0.0,0.0,3,"2,800","2,785",18
2022-09-30,0.0,0.0,18,"2,800","2,806",13


In [419]:
print("## Sales Account ##")
display(R(sales.df))

## Sales Account ##


,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.0,0,0,0,0,0
2022-01-31,0.0,0,0,"125,000",0,"125,000"
2022-02-28,0.0,0,"125,000",0,0,"125,000"
2022-03-31,0.0,0,"125,000",0,0,"125,000"
2022-04-30,0.0,0,"125,000",0,0,"125,000"
2022-05-31,0.0,0,"125,000",0,0,"125,000"
2022-06-30,0.0,0,"125,000",0,0,"125,000"
2022-07-31,0.0,0,"125,000",0,0,"125,000"
2022-08-31,0.0,0,"125,000",0,0,"125,000"
2022-09-30,0.0,0,"125,000",0,0,"125,000"


In [420]:
print("## Repayment Account ##")
display(R(acc.repay.df))

## Repayment Account ##


,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.0,0.0,0.0,0,0,0.0
2022-01-31,0.0,0.0,0.0,0,0,0.0
2022-02-28,0.0,0.0,0.0,0,0,0.0
2022-03-31,0.0,0.0,0.0,0,0,0.0
2022-04-30,0.0,0.0,0.0,0,0,0.0
2022-05-31,0.0,0.0,0.0,0,0,0.0
2022-06-30,0.0,0.0,0.0,0,0,0.0
2022-07-31,0.0,0.0,0.0,0,0,0.0
2022-08-31,0.0,0.0,0.0,0,0,0.0
2022-09-30,0.0,0.0,0.0,0,0,0.0


### 2) Costs

In [421]:
cost_dct = cost["all"].dct.items()
cost_df = DataFrame({key:item.amt_add[:] for key, item in cost_dct})
cost_df[("sum", "sum")] = cost["all"].amt_add[:]
display(R(cost_df))

lnd                                 cstrn cstrnidrt         \
             prchs brkrg tax_aqstn nhbond jdclcst  cstrn   wtrelec wsctbn   
2021-12-31   1,000  39.0         0    0.0     0.0     35         0      0   
2022-01-31  13,500  91.0       675   36.0    15.0  2,545         0      0   
2022-02-28       0   0.0         0    0.0     0.0  2,545         0      0   
2022-03-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-04-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-05-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-06-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-07-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-08-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-09-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-10-31       0   0.0         0    0.0     0.0  2,545         0      0   
2022-11-30       0   0.0         0    0.0     0.0  2,545         0      0   
2022-12-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-01-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-02-28       0   0.0         0    0.0     0.0  2,545         0      0   
2023-03-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-04-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-05-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-06-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-07-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-08-31       0   0.0         0    0.0     0.0  2,545         0      0   
2023-09-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-10-31       0   0.0         0    0.0     0.0  2,545       980    163   
2023-11-30       0   0.0         0    0.0     0.0  2,545         0      0   
2023-12-31       0   0.0         0    0.0     0.0      0         0      0   
2024-01-31       0   0.0         0    0.0     0.0      0         0      0   
2024-02-29       0   0.0         0    0.0     0.0      0         0      0   
2024-03-31       0   0.0         0    0.0     0.0      0         0      0   
2024-04-30       0   0.0         0    0.0     0.0      0         0      0   
2024-05-31       0   0.0         0    0.0     0.0      0         0      0   
2024-06-30       0   0.0         0    0.0     0.0  6,509         0      0   

                                     mrktg                     taxutlt  \
           dsgncst lcnscst spvsn rentagncy salescsltg advtsmnt prpttax   
2021-12-31       0       0   0.0         0          0        0     0.0   
2022-01-31     624     528  21.3         0          0        0     0.0   
2022-02-28       0       0  21.3         0          0        0     0.0   
2022-03-31       0       0  21.3         0          0        0     0.0   
2022-04-30       0       0  21.3         0          0        0     0.0   
2022-05-31       0       0  21.3         0          0        0     0.0   
2022-06-30       0       0  21.3         0          0      250    37.0   
2022-07-31       0       0  21.3         0          0        0     0.0   
2022-08-31       0       0  21.3         0          0        0     0.0   
2022-09-30       0       0  21.3         0          0        0     0.0   
2022-10-31       0       0  21.3         0          0        0     0.0   
2022-11-30       0       0  21.3         0          0        0     0.0   
2022-12-31       0       0  21.3         0          0        0     0.0   
2023-01-31       0       0  21.3         0          0        0     0.0   
2023-02-28       0       0  21.3         0          0        0     0.0   
2023-03-31       0       0  21.3       517          0        0     0.0   
2023-04-30       0       0  21.3       517          0        0     0.0   
2023-05-31       0       0  21.3       517          0        0     

### 3) Loan

#### 3-1) Notional Amount

In [431]:
loan_dct = loan.ttl.dct.items()

loan_df = {}
for key, item in loan_dct:
    loan_df[(key, 'amt_sub')] = item.ntnl.amt_sub[:]
    loan_df[(key, 'amt_add')] = item.ntnl.amt_add[:]
    loan_df[(key, 'bal_end')] = item.ntnl.bal_end[:]
loan_df = DataFrame(loan_df)
loan_df[('sum', 'amt_sub')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_sub"])
loan_df[('sum', 'amt_add')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_add"])
loan_df[('sum', 'bal_end')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "bal_end"])

display(R(loan_df))

tra                        trb                        trc  \
           amt_sub amt_add    bal_end amt_sub amt_add    bal_end amt_sub   
2021-12-31       0       0        0.0       0       0        0.0       0   
2022-01-31  15,000       0  -15,000.0  20,000       0  -20,000.0   5,000   
2022-02-28       0       0  -15,000.0       0       0  -20,000.0       0   
2022-03-31       0       0  -15,000.0       0       0  -20,000.0       0   
2022-04-30       0       0  -15,000.0       0       0  -20,000.0       0   
2022-05-31       0       0  -15,000.0       0       0  -20,000.0       0   
2022-06-30       0       0  -15,000.0       0       0  -20,000.0       0   
2022-07-31       0       0  -15,000.0       0       0  -20,000.0   1,400   
2022-08-31       0       0  -15,000.0       0       0  -20,000.0   2,800   
2022-09-30   2,000       0  -17,000.0       0       0  -20,000.0     800   
2022-10-31   2,900       0  -19,900.0       0       0  -20,000.0       0   
2022-11-30   2,800       0  -22,700.0       0       0  -20,000.0       0   
2022-12-31   2,800       0  -25,500.0       0       0  -20,000.0       0   
2023-01-31   2,900       0  -28,400.0       0       0  -20,000.0       0   
2023-02-28   2,900       0  -31,300.0       0       0  -20,000.0       0   
2023-03-31   3,300       0  -34,600.0       0       0  -20,000.0       0   
2023-04-30   3,400       0  -38,000.0       0       0  -20,000.0       0   
2023-05-31   3,400       0  -41,400.0       0       0  -20,000.0       0   
2023-06-30   4,100       0  -45,500.0       0       0  -20,000.0       0   
2023-07-31   2,900       0  -48,400.0       0       0  -20,000.0       0   
2023-08-31   3,000       0  -51,400.0       0       0  -20,000.0       0   
2023-09-30   2,900       0  -54,300.0       0       0  -20,000.0       0   
2023-10-31   4,100       0  -58,400.0       0       0  -20,000.0       0   
2023-11-30   6,000       0  -64,400.0       0       0  -20,000.0       0   
2023-12-31     600       0  -65,000.0       0       0  -20,000.0       0   
2024-01-31       0       0  -65,000.0       0       0  -20,000.0       0   
2024-02-29       0       0  -65,000.0       0       0  -20,000.0       0   
2024-03-31       0  65,000        0.0       0  20,000        0.0       0   
2024-04-30       0       0        0.0       0       0        0.0       0   
2024-05-31       0       0        0.0       0       0        0.0       0   
2024-06-30       0       0        0.0       0       0        0.0       0   

                                  sum                     
           amt_add    bal_end amt_sub amt_add    bal_end  
2021-12-31       0        0.0       0       0        0.0  
2022-01-31       0   -5,000.0  40,000       0  -40,000.0  
2022-02-28       0   -5,000.0       0       0  -40,000.0  
2022-03-31       0   -5,000.0       0       0  -40,000.0  
2022-04-30       0   -5,000.0       0       0  -40,000.0  
2022-05-31       0   -5,000.0       0       0  -40,000.0  
2022-06-30       0   -5,000.0       0       0  -40,000.0  
2022-07-31       0   -6,400.0   1,400       0  -41,400.0  
2022-08-31       0   -9,200.0   2,800       0  -44,200.0  
2022-09-30       0  -10,000.0   2,800       0  -47,000.0  
2022-10-31       0  -10,000.0   2,900       0  -49,900.0  
2022-11-30       0  -10,000.0   2,800       0  -52,700.0  
2022-12-31       0  -10,000.0   2,800       0  -55,500.0  
2023-01-31       0  -10,000.0   2,900       0  -58,400.0  
2023-02-28       0  -10,000.0   2,900       0  -61,300.0  
2023-03-31       0  -10,000.0   3,300       0  -64,600.0  
2023-04-30       0  -10,000.0   3,400       0  -68,000.0  
2023-05-31       0  -10,000.0   3,400       0  -71,400.0  
2023-06-30       0  -10,000.0   4,100       0  -75,500.0  
2023-07-31       0  -10,000.0   2,900       0  -78,400.0  
2023-08-31       0  -10,000.0   3,000       0  -81,400.0  
2023-09-30       0  -10,000.0   2,900       0  -84,300.0  
2023-10-31       0  -10,000.0   4,100       0  -88,400.0  
2023-11-30       0  -10,000.0  

#### 3-2) Fee Amount

In [465]:
R(DataFrame({key: sum(item.amt_add[:]) for key, item in loan.ttl.fee.dct.items()}, index=['amt_add']))

,tra,trb,trc
amt_add,975,700,700


In [432]:
loan_dct = loan.ttl.dct.items()

loan_df = {}
for key, item in loan_dct:
    loan_df[(key, 'amt_sub')] = item.fee.amt_sub[:]
    loan_df[(key, 'amt_add')] = item.fee.amt_add[:]
    loan_df[(key, 'bal_end')] = item.fee.bal_end[:]
loan_df = DataFrame(loan_df)
loan_df[('sum', 'amt_sub')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_sub"])
loan_df[('sum', 'amt_add')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_add"])
loan_df[('sum', 'bal_end')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "bal_end"])

display(R(loan_df))

tra                     trb                     trc          \
           amt_sub amt_add bal_end amt_sub amt_add bal_end amt_sub amt_add   
2021-12-31     0.0       0       0     0.0       0       0     0.0       0   
2022-01-31     0.0     975     975     0.0     700     700     0.0     700   
2022-02-28     0.0       0     975     0.0       0     700     0.0       0   
2022-03-31     0.0       0     975     0.0       0     700     0.0       0   
2022-04-30     0.0       0     975     0.0       0     700     0.0       0   
2022-05-31     0.0       0     975     0.0       0     700     0.0       0   
2022-06-30     0.0       0     975     0.0       0     700     0.0       0   
2022-07-31     0.0       0     975     0.0       0     700     0.0       0   
2022-08-31     0.0       0     975     0.0       0     700     0.0       0   
2022-09-30     0.0       0     975     0.0       0     700     0.0       0   
2022-10-31     0.0       0     975     0.0       0     700     0.0       0   
2022-11-30     0.0       0     975     0.0       0     700     0.0       0   
2022-12-31     0.0       0     975     0.0       0     700     0.0       0   
2023-01-31     0.0       0     975     0.0       0     700     0.0       0   
2023-02-28     0.0       0     975     0.0       0     700     0.0       0   
2023-03-31     0.0       0     975     0.0       0     700     0.0       0   
2023-04-30     0.0       0     975     0.0       0     700     0.0       0   
2023-05-31     0.0       0     975     0.0       0     700     0.0       0   
2023-06-30     0.0       0     975     0.0       0     700     0.0       0   
2023-07-31     0.0       0     975     0.0       0     700     0.0       0   
2023-08-31     0.0       0     975     0.0       0     700     0.0       0   
2023-09-30     0.0       0     975     0.0       0     700     0.0       0   
2023-10-31     0.0       0     975     0.0       0     700     0.0       0   
2023-11-30     0.0       0     975     0.0       0     700     0.0       0   
2023-12-31     0.0       0     975     0.0       0     700     0.0       0   
2024-01-31     0.0       0     975     0.0       0     700     0.0       0   
2024-02-29     0.0       0     975     0.0       0     700     0.0       0   
2024-03-31     0.0       0     975     0.0       0     700     0.0       0   
2024-04-30     0.0       0     975     0.0       0     700     0.0       0   
2024-05-31     0.0       0     975     0.0       0     700     0.0       0   
2024-06-30     0.0       0     975     0.0       0     700     0.0       0   

                       sum                  
           bal_end amt_sub amt_add bal_end  
2021-12-31       0     0.0       0       0  
2022-01-31     700     0.0   2,375   2,375  
2022-02-28     700     0.0       0   2,375  
2022-03-31     700     0.0       0   2,375  
2022-04-30     700     0.0       0   2,375  
2022-05-31     700     0.0       0   2,375  
2022-06-30     700     0.0       0   2,375  
2022-07-31     700     0.0       0   2,375  
2022-08-31     700     0.0       0   2,375  
2022-09-30     700     0.0       0   2,375  
2022-10-31     700     0.0       0   2,375  
2022-11-30     700     0.0       0   2,375  
2022-12-31     700     0.0       0   2,375  
2023-01-31     700     0.0       0   2,375  
2023-02-28     700     0.0       0   2,375  
2023-03-31     700     0.0       0   2,375  
2023-04-30     700     0.0       0   2,375  
2023-05-31     700     0.0       0   2,375  
2023-06-30     700     0.0       0   2,375  
2023-07-31     700     0.0       0   2,375  
2023-08-31     700     0.0       0   2,375  
2023-09-30     700     0.0       0   2,375  
2023-10-31     700     0.0       0   2,375  
2023-11-30     700     0.0       0   2,375  
2023-12-31     700     0.0       0   2,375  
2024-01-31     700     0.0       0   2,375  
2024-02-29     700     0.0       0   2,375  
2024-03-31     700     0.0       0   2,375  
2024-04-30     700     0.0       0   2,375  
2024-05-31     700     0.0       0   2,375  


#### 3-3) IR Amount

In [464]:
R(DataFrame({key: sum(item.amt_add[:]) for key, item in loan.ttl.IR.dct.items()}, index=['amt_add']))

,tra,trb,trc
amt_add,"2,987","2,600","1,692"


In [433]:
loan_dct = loan.ttl.dct.items()

loan_df = {}
for key, item in loan_dct:
    loan_df[(key, 'amt_sub')] = item.IR.amt_sub[:]
    loan_df[(key, 'amt_add')] = item.IR.amt_add[:]
    loan_df[(key, 'bal_end')] = item.IR.bal_end[:]
loan_df = DataFrame(loan_df)
loan_df[('sum', 'amt_sub')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_sub"])
loan_df[('sum', 'amt_add')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_add"])
loan_df[('sum', 'bal_end')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "bal_end"])

display(R(loan_df))

tra                     trb                     trc          \
           amt_sub amt_add bal_end amt_sub amt_add bal_end amt_sub amt_add   
2021-12-31     0.0       0       0     0.0       0       0     0.0     0.0   
2022-01-31     0.0       0       0     0.0       0       0     0.0     0.0   
2022-02-28     0.0      50      50     0.0     100     100     0.0    37.5   
2022-03-31     0.0      50     100     0.0     100     200     0.0    37.5   
2022-04-30     0.0      50     150     0.0     100     300     0.0    37.5   
2022-05-31     0.0      50     200     0.0     100     400     0.0    37.5   
2022-06-30     0.0      50     250     0.0     100     500     0.0    37.5   
2022-07-31     0.0      50     300     0.0     100     600     0.0    37.5   
2022-08-31     0.0      50     350     0.0     100     700     0.0    48.0   
2022-09-30     0.0      50     400     0.0     100     800     0.0    69.0   
2022-10-31     0.0      57     457     0.0     100     900     0.0    75.0   
2022-11-30     0.0      66     523     0.0     100   1,000     0.0    75.0   
2022-12-31     0.0      76     599     0.0     100   1,100     0.0    75.0   
2023-01-31     0.0      85     684     0.0     100   1,200     0.0    75.0   
2023-02-28     0.0      95     778     0.0     100   1,300     0.0    75.0   
2023-03-31     0.0     104     883     0.0     100   1,400     0.0    75.0   
2023-04-30     0.0     115     998     0.0     100   1,500     0.0    75.0   
2023-05-31     0.0     127   1,125     0.0     100   1,600     0.0    75.0   
2023-06-30     0.0     138   1,263     0.0     100   1,700     0.0    75.0   
2023-07-31     0.0     152   1,414     0.0     100   1,800     0.0    75.0   
2023-08-31     0.0     161   1,576     0.0     100   1,900     0.0    75.0   
2023-09-30     0.0     171   1,747     0.0     100   2,000     0.0    75.0   
2023-10-31     0.0     181   1,928     0.0     100   2,100     0.0    75.0   
2023-11-30     0.0     195   2,123     0.0     100   2,200     0.0    75.0   
2023-12-31     0.0     215   2,337     0.0     100   2,300     0.0    75.0   
2024-01-31     0.0     217   2,554     0.0     100   2,400     0.0    75.0   
2024-02-29     0.0     217   2,771     0.0     100   2,500     0.0    75.0   
2024-03-31     0.0     217   2,987     0.0     100   2,600     0.0    75.0   
2024-04-30     0.0       0   2,987     0.0       0   2,600     0.0     0.0   
2024-05-31     0.0       0   2,987     0.0       0   2,600     0.0     0.0   
2024-06-30     0.0       0   2,987     0.0       0   2,600     0.0     0.0   

                       sum                  
           bal_end amt_sub amt_add bal_end  
2021-12-31       0     0.0       0       0  
2022-01-31       0     0.0       0       0  
2022-02-28      38     0.0     188     188  
2022-03-31      75     0.0     188     375  
2022-04-30     112     0.0     188     562  
2022-05-31     150     0.0     188     750  
2022-06-30     188     0.0     188     938  
2022-07-31     225     0.0     188   1,125  
2022-08-31     273     0.0     198   1,323  
2022-09-30     342     0.0     219   1,542  
2022-10-31     417     0.0     232   1,774  
2022-11-30     492     0.0     241   2,015  
2022-12-31     567     0.0     251   2,266  
2023-01-31     642     0.0     260   2,526  
2023-02-28     717     0.0     270   2,795  
2023-03-31     792     0.0     279   3,075  
2023-04-30     867     0.0     290   3,365  
2023-05-31     942     0.0     302   3,667  
2023-06-30   1,017     0.0     313   3,980  
2023-07-31   1,092     0.0     327   4,306  
2023-08-31   1,167     0.0     336   4,643  
2023-09-30   1,242     0.0     346   4,989  
2023-10-31   1,317     0.0     356   5,345  
2023-11-30   1,392     0.0     370   5,715  
2023-12-31   1,467     0.0     390   6,104  
2024-01-31   1,542     0.0     392   6,496  
2024-02-29   1,617     0.0     392   6,888  
2024-03-31   1,692     0.0     392   7,279  
2024-04-30   1,692     0.0       0   7,279  
2024-05-31   1,692     0.0       0   7,279  
